In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [9]:
# Constants
DATA_DIR = '/content/drive/MyDrive/JPEGImages'
BATCH_SIZE = 64
NUM_WORKERS = 8
PIN_MEMORY = True
TRAINING_EPOCHS = 10
LEARNING_RATE = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Loading and preprocessing
def load_data(data_dir):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize all images to 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = ImageFolder(data_dir, transform=transform)
    class_counts = np.array([0] * len(dataset.classes))
    for _, index in dataset.samples:
        class_counts[index] += 1
    class_weights = 1. / (torch.tensor(class_counts, dtype=torch.float) + 1e-6)
    sample_weights = class_weights[[index for _, index in dataset.samples]]
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_indices = [dataset.samples[i][1] for i in train_dataset.indices]  # get class indices for training samples
    train_sampler = WeightedRandomSampler(class_weights[train_indices], len(train_indices))
    return train_dataset, test_dataset, train_sampler

train_data, test_data, train_sampler = load_data(DATA_DIR)

# DataLoader
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

In [10]:
# Model definition
def create_model():
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.BatchNorm1d(512),
        nn.Linear(512, 50)
    )
    return model

model = create_model()
model = model.to(device)


In [14]:
# Model Training
def train_model(model, train_loader):
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scaler = torch.cuda.amp.GradScaler()
    model.train()
    for epoch in range(TRAINING_EPOCHS):
        start_time = time.time()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if i % 20 == 0:
                print(f'Epoch {epoch}, Step {i}, Loss: {loss.item()}')
        epoch_duration = time.time() - start_time
        print(f'Epoch {epoch} completed in {epoch_duration:.2f} seconds')


In [15]:
# Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')



In [16]:
# Run training and evaluation
train_model(model, train_loader)


Epoch 0, Step 0, Loss: 2.119572639465332
Epoch 0, Step 20, Loss: 2.3155622482299805
Epoch 0, Step 40, Loss: 2.225698947906494
Epoch 0, Step 60, Loss: 1.9682159423828125
Epoch 0, Step 80, Loss: 1.8339200019836426
Epoch 0, Step 100, Loss: 1.5913962125778198
Epoch 0, Step 120, Loss: 1.7068402767181396
Epoch 0 completed in 28.12 seconds
Epoch 1, Step 0, Loss: 1.3185474872589111
Epoch 1, Step 20, Loss: 1.0905601978302002
Epoch 1, Step 40, Loss: 1.4415557384490967
Epoch 1, Step 60, Loss: 1.6738741397857666
Epoch 1, Step 80, Loss: 1.4883043766021729
Epoch 1, Step 100, Loss: 1.3111207485198975
Epoch 1, Step 120, Loss: 1.1752679347991943
Epoch 1 completed in 28.16 seconds
Epoch 2, Step 0, Loss: 1.0407236814498901
Epoch 2, Step 20, Loss: 1.2859675884246826
Epoch 2, Step 40, Loss: 1.452837347984314
Epoch 2, Step 60, Loss: 1.2161704301834106
Epoch 2, Step 80, Loss: 1.0208377838134766
Epoch 2, Step 100, Loss: 1.2581877708435059
Epoch 2, Step 120, Loss: 0.9104627966880798
Epoch 2 completed in 28.19 

In [17]:
evaluate_model(model, test_loader)

Accuracy on the test set: 64.93%
